In [20]:
from TN import TN
# from pydoc import locate
import os
import utils
import torchvision, torch
import numpy as np
import pickle
from sklearn.metrics.pairwise import euclidean_distances
# import argparse, pickle
# model_ckpt = '/net/scratch/chacha/explain_teach/models/results/chacha-syn-htriplets/2ul5cslk/checkpoints/epoch=117-valid_loss=0.00.ckpt'
# model_ckpt = '/net/scratch/chacha/explain_teach/models/results/synthetic_MTL/3sk1rynk/checkpoints/best_model.ckpt.'

# args = argparse.Namespace(embed_dim=10)
# model = TN(args)

# model = model.load_from_checkpoint(model_ckpt, **vars(args)).to("cuda")
# model.eval()


# dict_args = {'gpus': -1, 'seed': 42, 'max_epochs': 200, 'learning_rate': 0.0001, 'train_batch_size': 120, 'eval_batch_size': 64, 'dataloader_num_workers': 4, 'num_class': 2, 'train_dir': '/net/scratch/chacha/data/weevil_vespula/train', 'valid_dir': '/net/scratch/chacha/data/weevil_vespula/valid', 'test_dir': '/net/scratch/chacha/data/weevil_vespula/test', 'wandb_group': 'unpretrained', 'wandb_mode': 'online', 'wandb_project': 'chacha-syn-htriplets', 'wandb_entity': 'ai-driven-tutorial', 'wandb_name': 'TN_syn', 'do_train': True, 'do_test': True, 'pretrained': False, 'embed_dim': 10, 'transform': 'wv', 'train_triplets': '/net/scratch/chacha/data/weevil_vespula/train_triplet.pkl', 'valid_triplets': '/net/scratch/chacha/data/weevil_vespula/valid_triplet.pkl', 'test_triplets': '/net/scratch/chacha/data/weevil_vespula/test_triplet.pkl', 'subset': False}
# model = TN(**dict_args)

# model = MyLightningModule(hparams)
# new_model = TN.load_from_checkpoint(checkpoint_path=model_ckpt)
# model = locate(model_ckpt)
# model = model.load_from_checkpoint(model_ckpt, **vars(args)).to("cuda")

In [50]:

data_parent_dir = "/net/scratch/chacha/data/weevil_vespula"
split = 'train'
data_dir = os.path.join(data_parent_dir,split)
transform = utils.get_transform('wv', aug=False)
dataset = torchvision.datasets.ImageFolder(data_dir, transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=len(dataset), num_workers=4)
groundtruth_train_labels = list(iter(dataloader))[0][1].numpy()

split = 'test'
data_dir = os.path.join(data_parent_dir,split)
transform = utils.get_transform('wv', aug=False)
dataset = torchvision.datasets.ImageFolder(data_dir, transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=len(dataset), num_workers=4)
groundtruth_test_labels = list(iter(dataloader))[0][1].numpy()


# if len(dataset) <= 128:
#     dataloader = torch.utils.data.DataLoader(dataset, batch_size=len(dataset), num_workers=4)
#     embeds = model.embed(list(iter(dataloader))[0][0].cuda())
#     embeds = embeds.cpu().detach().numpy()
# else:
#     dataloader = torch.utils.data.DataLoader(dataset, batch_size=128, num_workers=4)

## 1NN accuracy

In [16]:
wv_tn_train = np.array(pickle.load(open("../embeds/wv/TN_train_emb10.pkl", "rb")))
wv_tn_valid = np.array(pickle.load(open("../embeds/wv/TN_valid_emb10.pkl", "rb")))
wv_tn_test = np.array(pickle.load(open("../embeds/wv/TN_test_emb10.pkl", "rb")))


In [22]:
## for each test embeds, find nearest neighbor in train embeds retrieve the nearest neighbor label
# for test in wv_tn_test:
#     print(test)
#     dists = euclidean_distances(test, wv_tn_train)
#     print(dists)
#     # np.argsort(dist[index])[1]
#     # find_1nn(wv_tn_train,test)
#     break
dists = euclidean_distances(wv_tn_test, wv_tn_train)

In [54]:
# dists[0][120]
# groundtruth_labels
# np.argmin(dists[0], axis=0)
# min(dists[0])

In [53]:
acc = 0
for i in range(dists.shape[0]):
    ##iterate over 40 test examples to retrieve the nearest neighbor
    min_index = np.argmin(dists[i], axis=0)
    this_label = groundtruth_train_labels[min_index]
    test_label = groundtruth_test_labels[i] ##
    if this_label == test_label:
        acc += 1
acc/dists.shape[0]

0.925

## Triplets acc

In [67]:
triplet_file_path = '/net/scratch/chacha/data/weevil_vespula/test_triplet.pkl'
wv_test_triplet = np.array(pickle.load(open(triplet_file_path, "rb")))
def euc_dist(x, y): return np.sqrt(np.dot(x, x) - 2 * np.dot(x, y) + np.dot(y, y))

In [78]:
trip_acc = 0
for trip in wv_test_triplet:
    a,p,n = trip
    dist_ap = euc_dist(wv_tn_test[a],wv_tn_test[p])
    dist_an = euc_dist(wv_tn_test[a],wv_tn_test[n])
    if dist_ap <= dist_an:
        trip_acc +=1
    # break
trip_acc/wv_test_triplet.shape[0]

0.9174

In [79]:
# print(a,p,n)
# print(dist_ap,dist_an)
# print(wv_tn_test[a])
# print(wv_tn_test[p])
# print(wv_tn_test[n])
trip_acc

9174

In [80]:
wv_test_triplet.shape

(10000, 3)